In [2]:
# Import necessary libraries
import pandas as pd

# Load SafeGraph LA foot traffic data
df_foot_traffic = pd.read_csv('/Users/darpanradadiya/Downloads/TESLA_CHARGER_DASHBOARD/data/la_foot_traffic.csv')

# Inspect first few rows and data types
df_foot_traffic.head(), df_foot_traffic.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128908 entries, 0 to 128907
Data columns (total 52 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   placekey                                    128908 non-null  object 
 1   parent_placekey                             18080 non-null   object 
 2   safegraph_brand_ids                         9516 non-null    object 
 3   location_name                               128908 non-null  object 
 4   brands                                      9516 non-null    object 
 5   store_id                                    10071 non-null   object 
 6   top_category                                128908 non-null  object 
 7   sub_category                                105217 non-null  object 
 8   naics_code                                  128908 non-null  int64  
 9   latitude                                    128908 non-null  float64
 

(              placekey parent_placekey safegraph_brand_ids  \
 0  22h-222@5z4-zwd-ffz             NaN                 NaN   
 1  22h-222@5z4-zw9-syv             NaN                 NaN   
 2  zzy-223@5z5-3rs-k75             NaN                 NaN   
 3  224-224@5z5-3qs-cqz             NaN                 NaN   
 4  225-222@5z6-3py-9s5             NaN                 NaN   
 
              location_name brands store_id  \
 0               MagIQ Room    NaN      NaN   
 1       DormShare Westwood    NaN      NaN   
 2            Ace Hotel Inn    NaN      NaN   
 3             888 Wilshire    NaN      NaN   
 4  Jim Dandy Fried Chicken    NaN      NaN   
 
                                 top_category  \
 0  Other Amusement and Recreation Industries   
 1                     Traveler Accommodation   
 2                     Traveler Accommodation   
 3        Restaurants and Other Eating Places   
 4        Restaurants and Other Eating Places   
 
                                     sub

In [3]:
# Show the most common top_category and sub_category values
print("Top Categories:\n", df_foot_traffic['top_category']
      .value_counts().head(20), "\n")
print("Sub Categories:\n", df_foot_traffic['sub_category']
      .value_counts().head(20))


Top Categories:
 top_category
Restaurants and Other Eating Places                               14696
Urban Transit Systems                                              8425
Legal Services                                                     8275
Personal Care Services                                             6786
Clothing Stores                                                    5883
Offices of Other Health Practitioners                              4117
Religious Organizations                                            3953
Offices of Physicians                                              3636
Automotive Repair and Maintenance                                  3335
Jewelry, Luggage, and Leather Goods Stores                         3144
Grocery Stores                                                     2299
Health and Personal Care Stores                                    2295
Other Amusement and Recreation Industries                          2251
Gasoline Stations                 

In [4]:
# Define upscale categories
upscale_cats = [
    "Restaurants and Other Eating Places",     # full-service restaurants
    "Jewelry, Luggage, and Leather Goods Stores",
    "Fitness and Recreational Sports Centers"
]

# Filter the foot traffic data
df_upscale = df_foot_traffic[
    df_foot_traffic["top_category"].isin(upscale_cats)
].copy()

# Quick check
print("Filtered entries:", len(df_upscale))
df_upscale["top_category"].value_counts()


Filtered entries: 17840


top_category
Restaurants and Other Eating Places           14696
Jewelry, Luggage, and Leather Goods Stores     3144
Name: count, dtype: int64

In [5]:
# Drop entries without visit counts or dwell time
df_scoring = df_upscale.dropna(subset=['raw_visit_counts', 'median_dwell']).copy()

# Verify how many remain
print("Entries with complete traffic & dwell data:", len(df_scoring))


Entries with complete traffic & dwell data: 8631


In [6]:
from sklearn.preprocessing import MinMaxScaler

# Initialize scaler and fit-transform
scaler = MinMaxScaler()
df_scoring[['visit_score', 'dwell_score']] = scaler.fit_transform(
    df_scoring[['raw_visit_counts', 'median_dwell']]
)

# Compute composite suitability score
df_scoring['suitability_score'] = (
    0.6 * df_scoring['visit_score'] +
    0.4 * df_scoring['dwell_score']
)

# Preview results
df_scoring[['location_name', 'visit_score', 'dwell_score', 'suitability_score']].head()


,location_name,visit_score,dwell_score,suitability_score
47,The Beverly Hills Bagel,0.0,0.067577,0.027031
48,Oscarito's Catering,0.0,0.017973,0.007189
49,Taste of Thai,0.0,0.009346,0.003738
50,Swami's Sandwiches Food Truck,0.0,0.048886,0.019554
51,Gogo's Tacos,0.0,0.009346,0.003738


In [10]:
import requests
import pandas as pd

# Fetch ACS data for total households and households with no vehicles
census_url = "https://api.census.gov/data/2022/acs/acs5"
params = {
    "get": "B08201_001E,B08201_002E,NAME",  # total households, no‐vehicle households
    "for": "zip code tabulation area:*"
}
resp = requests.get(census_url, params=params)
data = resp.json()

# Build DataFrame
cols = data[0]
df_census = pd.DataFrame(data[1:], columns=cols)
df_census[['B08201_001E','B08201_002E']] = df_census[['B08201_001E','B08201_002E']].astype(int)

# Compute vehicle‐available households as proxy
df_census['veh_households'] = df_census['B08201_001E'] - df_census['B08201_002E']
df_census = df_census.rename(columns={'zip code tabulation area':'zip'})

# Inspect LA ZIPs subset (e.g., 90001–91609)
la_zips = df_census[df_census['zip'].astype(int).between(90001,91609)]
la_zips[['zip','veh_households']].head()


,zip,veh_households
30597,90001,12335
30598,90002,11362
30599,90003,15276
30600,90004,19152
30601,90005,11873


In [11]:
# Ensure ZIP code columns align
df_census['zip'] = df_census['zip'].astype(int)
df_scoring['postal_code'] = df_scoring['postal_code'].astype(int)

# Merge on ZIP/postal_code
df_merged = df_scoring.merge(
    df_census[['zip', 'veh_households']],
    left_on='postal_code',
    right_on='zip',
    how='left'
)

# Check merge
print("Merged entries:", len(df_merged))
df_merged[['location_name', 'postal_code', 'veh_households']].head()


Merged entries: 8631


,location_name,postal_code,veh_households
0,The Beverly Hills Bagel,90035,10405.0
1,Oscarito's Catering,90021,687.0
2,Taste of Thai,90028,12499.0
3,Swami's Sandwiches Food Truck,90015,9767.0
4,Gogo's Tacos,90004,19152.0


In [12]:
from sklearn.preprocessing import MinMaxScaler

# Normalize veh_households
scaler_ev = MinMaxScaler()
df_merged['ev_score'] = scaler_ev.fit_transform(
    df_merged[['veh_households']].fillna(0)
)

# Preview
df_merged[['location_name', 'veh_households', 'ev_score']].head()


,location_name,veh_households,ev_score
0,The Beverly Hills Bagel,10405.0,0.343842
1,Oscarito's Catering,687.0,0.022702
2,Taste of Thai,12499.0,0.413040
3,Swami's Sandwiches Food Truck,9767.0,0.322759
4,Gogo's Tacos,19152.0,0.632894


In [13]:
# Compute final composite score
df_merged['final_score'] = (
    0.6 * df_merged['suitability_score'] +
    0.4 * df_merged['ev_score']
)

# Show top 10 locations by final_score
top10 = df_merged.nlargest(10, 'final_score')
top10[['location_name', 'postal_code', 'suitability_score', 'ev_score', 'final_score']]


,location_name,postal_code,suitability_score,ev_score,final_score
1858,Kushi,90026,0.389460,0.759459,0.537460
680,Juicy Lucy's,90033,0.600000,0.366478,0.506591
6848,Tacos Yorba's,90731,0.357960,0.688180,0.490048
2678,Taco Man,90004,0.349909,0.632894,0.463103
3756,Bagelworks Cafe,90025,0.318459,0.663329,0.456407
4585,Melo Dines,90025,0.281795,0.663329,0.434409
6502,Mama Hong's Vietnamese Kitchen West Los Angeles,90025,0.274628,0.663329,0.430108
771,Unity LA,90045,0.369869,0.504213,0.423606
8239,Century Restaurant,90045,0.369869,0.504213,0.423606
7728,Bontika,90036,0.293546,0.597667,0.415195


In [15]:
# Install OSMnx if needed
!pip install osmnx

import osmnx as ox

# Download drivable road network for Los Angeles
place_name = "Los Angeles, California, USA"
G = ox.graph_from_place(place_name, network_type="drive")

# Convert edges (roads) to GeoDataFrame
gdf_roads = ox.graph_to_gdfs(G, nodes=False, edges=True)

# Inspect
gdf_roads.head()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.5/100.5 kB 3.3 MB/s eta 0:00:00


osmid  \
u      v          key                                                  
653688 21300195   0     [907145138, 895315641, 895315642, 398770659]   
       1614655105 0    [1376721881, 398770658, 759468526, 759468527]   
653689 1711811908 0                                        398771138   
       122734100  0                           [404964730, 398771139]   
       1711811906 0                                        643327598   

                         highway      lanes maxspeed                name  \
u      v          key                                                      
653688 21300195   0    secondary          6   35 mph  National Boulevard   
       1614655105 0    secondary  [4, 6, 5]   35 mph  National Boulevard   
653689 1711811908 0    secondary          5   35 mph  National Boulevard   
       122734100  0    secondary     [4, 5]   35 mph  National Boulevard   
       1711811906 0     tertiary        NaN      NaN     Military Avenue   

                       oneway reversed      length  \
u      v          key                                
653688 21300195   0     False    False   88.746024   
       1614655105 0     False     True  102.260072   
653689 1711811908 0     False    False   19.377863   
       122734100  0     False     True  109.728889   
       1711811906 0     False    False   19.493944   

                                                                geometry  \
u      v          key                                                      
653688 21300195   0    LINESTRING (-118.42955 34.02702, -118.42948 34...   
       1614655105 0    LINESTRING (-118.42955 34.02702, -118.4298 34....   
653689 1711811908 0    LINESTRING (-118.42547 34.02864, -118.42538 34...   
       122734100  0    LINESTRING (-118.42547 34.02864, -118.42556 34...   
       1711811906 0    LINESTRING (-118.42547 34.02864, -118.42554 34...   

                      bridge  ref access tunnel width junction  
u      v          key                                           
653688 21300195   0      NaN  NaN    NaN    NaN   NaN      NaN  
       1614655105 0      NaN  NaN    NaN    NaN   NaN      NaN  
653689 1711811908 0      NaN  NaN    NaN    NaN   NaN      NaN  
       122734100  0      NaN  NaN    NaN    NaN   NaN      NaN  
       1711811906 0      NaN  NaN    NaN    NaN   NaN      NaN

In [16]:
import geopandas as gpd
from shapely.geometry import Point

# Create GeoDataFrame for POIs
gdf_pois = gpd.GeoDataFrame(
    df_merged,
    geometry=[Point(xy) for xy in zip(df_merged.longitude, df_merged.latitude)],
    crs="EPSG:4326"
)

# Reproject both to a projected CRS for accurate distance (e.g., UTM zone 11N)
gdf_pois = gdf_pois.to_crs(epsg=32611)
gdf_roads_proj = gdf_roads.to_crs(epsg=32611)

# Compute nearest road distance
gdf_pois['dist_to_road'] = gdf_pois.geometry.apply(
    lambda pt: gdf_roads_proj.distance(pt).min()
)

# Preview
gdf_pois[['location_name', 'dist_to_road']].head()


,location_name,dist_to_road
0,The Beverly Hills Bagel,24.320335
1,Oscarito's Catering,21.059996
2,Taste of Thai,15.546258
3,Swami's Sandwiches Food Truck,34.518358
4,Gogo's Tacos,16.577585


In [17]:
# Invert and normalize distance (closer = better)
gdf_pois['access_score'] = 1 - MinMaxScaler().fit_transform(
    gdf_pois[['dist_to_road']]
)

# Preview accessibility score
gdf_pois[['location_name', 'dist_to_road', 'access_score']].head()


,location_name,dist_to_road,access_score
0,The Beverly Hills Bagel,24.320335,0.998181
1,Oscarito's Catering,21.059996,0.998425
2,Taste of Thai,15.546258,0.998838
3,Swami's Sandwiches Food Truck,34.518358,0.997418
4,Gogo's Tacos,16.577585,0.998761


In [18]:
# Compute final integrated suitability score
gdf_pois['integrated_score'] = (
    0.5 * gdf_pois['suitability_score'] +
    0.3 * gdf_pois['ev_score'] +
    0.2 * gdf_pois['access_score']
)

# Show top 10 final recommendations
final_top10 = gdf_pois.nlargest(10, 'integrated_score')
final_top10[['location_name', 'integrated_score']]


,location_name,integrated_score
1858,Kushi,0.622406
680,Juicy Lucy's,0.609930
2678,Taco Man,0.564378
3756,Bagelworks Cafe,0.557841
4585,Melo Dines,0.539533
6502,Mama Hong's Vietnamese Kitchen West Los Angeles,0.535583
771,Unity LA,0.535497
8239,Century Restaurant,0.535492
7728,Bontika,0.525576
7698,Brasserie,0.519957


In [19]:
# Save final recommendations to CSV
final_top10.to_csv('../data/top10_tesla_charger_sites.csv', index=False)

print("Final recommendations saved!")


Final recommendations saved!


In [21]:
# Filter existing Tesla charger locations
tesla_chargers = df_foot_traffic[df_foot_traffic['brands'].str.contains('Tesla', na=False)].copy()

# Convert to GeoDataFrame
gdf_tesla_chargers = gpd.GeoDataFrame(
    tesla_chargers,
    geometry=gpd.points_from_xy(tesla_chargers.longitude, tesla_chargers.latitude),
    crs="EPSG:4326"
).to_crs(epsg=32611)

# Inspect Tesla charger locations
print(gdf_tesla_chargers[['location_name', 'geometry']])


                    location_name                        geometry
410     Tesla Destination Charger   POINT (384578.84 3768690.266)
689     Tesla Destination Charger  POINT (385308.212 3768455.354)
1281    Tesla Destination Charger  POINT (366670.727 3768163.516)
3241    Tesla Destination Charger  POINT (377727.583 3774071.195)
5590    Tesla Destination Charger  POINT (384118.998 3768680.261)
8594    Tesla Destination Charger  POINT (375934.883 3770914.604)
9949    Tesla Destination Charger  POINT (371588.027 3774814.549)
10064   Tesla Destination Charger   POINT (373520.82 3773642.579)
12769   Tesla Destination Charger    POINT (366396.497 3769591.2)
13974   Tesla Destination Charger  POINT (383125.987 3767994.457)
17180   Tesla Destination Charger  POINT (372311.297 3757011.838)
21213          Tesla Supercharger  POINT (369054.235 3769607.419)
27118   Tesla Destination Charger  POINT (377205.564 3767731.972)
27915   Tesla Destination Charger  POINT (383977.395 3768432.795)
29906   Te

In [22]:
# Compute distance to nearest Tesla charger for each POI
gdf_pois['dist_to_tesla'] = gdf_pois.geometry.apply(
    lambda pt: gdf_tesla_chargers.geometry.distance(pt).min()
)

# Normalize distance as proximity_score (greater distance = higher score)
gdf_pois['proximity_score'] = MinMaxScaler().fit_transform(
    gdf_pois[['dist_to_tesla']]
)

# Preview results
gdf_pois[['location_name', 'dist_to_tesla', 'proximity_score']].head()



,location_name,dist_to_tesla,proximity_score
0,The Beverly Hills Bagel,1061.831948,0.041487
1,Oscarito's Catering,418.764618,0.016355
2,Taste of Thai,644.884684,0.025192
3,Swami's Sandwiches Food Truck,630.451510,0.024628
4,Gogo's Tacos,1470.206100,0.057447


In [23]:
# Final updated integrated score including proximity
gdf_pois['final_score_v2'] = (
    0.4 * gdf_pois['suitability_score'] +
    0.25 * gdf_pois['ev_score'] +
    0.2 * gdf_pois['access_score'] +
    0.15 * gdf_pois['proximity_score']
)

# Show updated top 10
top10_updated = gdf_pois.nlargest(10, 'final_score_v2')
top10_updated[['location_name', 'final_score_v2']]


,location_name,final_score_v2
1858,Kushi,0.548859
680,Juicy Lucy's,0.545460
1025,DMC Raven,0.507935
2678,Taco Man,0.507334
3756,Bagelworks Cafe,0.499540
6192,Tacos El Pimi,0.496729
7032,Alfredo's Mexican Food,0.495496
4585,Melo Dines,0.479520
8239,Century Restaurant,0.479316
771,Unity LA,0.479060


In [24]:
# Save top-ranked POIs with final_score_v2 to CSV
gdf_pois[['location_name', 'latitude', 'longitude', 'final_score_v2']].sort_values(
    by='final_score_v2', ascending=False
).to_csv('../data/final_recommendations.csv', index=False)

# Optionally, save with geometry to GeoJSON (for maps)
gdf_pois[['location_name', 'final_score_v2', 'geometry']].to_file(
    '../data/final_recommendations.geojson', driver='GeoJSON'
)

print("Export complete!")


Export complete!


In [26]:
import osmnx as ox

# Get parking geometries
gdf_parking = ox.features_from_place(
    "Los Angeles, California, USA",
    tags={"amenity": "parking"}
)

# Drop null geometry and reproject
gdf_parking = gdf_parking[gdf_parking.geometry.notnull()]
gdf_parking = gdf_parking.to_crs(epsg=32611)

# Preview
gdf_parking[['geometry']].head()


geometry
element id                                       
node    156995093  POINT (370050.306 3756876.341)
        243645428  POINT (374586.066 3762900.949)
        267057015  POINT (369614.627 3775883.584)
        305530048  POINT (364152.628 3783100.865)
        338837987   POINT (370064.66 3756747.421)

In [27]:
from sklearn.preprocessing import MinMaxScaler

# Compute distance from each POI to nearest parking lot
gdf_pois['dist_to_parking'] = gdf_pois.geometry.apply(
    lambda pt: gdf_parking.geometry.distance(pt).min()
)

# Normalize as parking score (closer = better)
gdf_pois['parking_score'] = 1 - MinMaxScaler().fit_transform(
    gdf_pois[['dist_to_parking']]
)

# Preview
gdf_pois[['location_name', 'dist_to_parking', 'parking_score']].head()


,location_name,dist_to_parking,parking_score
0,The Beverly Hills Bagel,352.299701,0.947073
1,Oscarito's Catering,5.885649,0.999116
2,Taste of Thai,23.282192,0.996502
3,Swami's Sandwiches Food Truck,147.527743,0.977837
4,Gogo's Tacos,38.128796,0.994272


In [28]:
# Final integrated score with parking
gdf_pois['final_score_v3'] = (
    0.35 * gdf_pois['suitability_score'] +
    0.25 * gdf_pois['ev_score'] +
    0.15 * gdf_pois['access_score'] +
    0.15 * gdf_pois['proximity_score'] +
    0.10 * gdf_pois['parking_score']
)

# Show updated top 10
top10_v3 = gdf_pois.nlargest(10, 'final_score_v3')
top10_v3[['location_name', 'final_score_v3']]


,location_name,final_score_v3
1858,Kushi,0.579030
680,Juicy Lucy's,0.559713
1025,DMC Raven,0.547784
6848,Tacos Yorba's,0.539849
2678,Taco Man,0.537925
6192,Tacos El Pimi,0.536619
7032,Alfredo's Mexican Food,0.534897
3756,Bagelworks Cafe,0.531483
6310,Holla Halo,0.516391
4874,My Mango Sticky Rice,0.516376


In [30]:
gdf_final = gdf_pois[['location_name', 'final_score_v3', 'geometry']].copy()


In [32]:
# Export to GeoJSON for Streamlit use
gdf_final.to_file("../data/final_scored_locations.geojson", driver="GeoJSON")


In [34]:
# Final export of all scoring columns to GeoJSON
gdf_pois[[
    'location_name', 'latitude', 'longitude',
    'suitability_score', 'ev_score', 'access_score',
    'proximity_score', 'parking_score', 'final_score_v3', 'geometry'
]].to_file("../data/final_scored_locations.geojson", driver="GeoJSON")


In [36]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Load LA foot traffic data
df = pd.read_csv("../data/la_foot_traffic.csv")

# Filter current Tesla chargers
tesla_chargers = df[df['brands'].str.contains("Tesla Destination Charger", na=False)].copy()

# Convert to GeoDataFrame
gdf_tesla = gpd.GeoDataFrame(
    tesla_chargers,
    geometry=gpd.points_from_xy(tesla_chargers.longitude, tesla_chargers.latitude),
    crs="EPSG:4326"
)

# Export to GeoJSON
gdf_tesla.to_file("../data/current_tesla_chargers.geojson", driver="GeoJSON")


In [37]:
# %%
import ipywidgets as widgets
from IPython.display import display

# Prepare the two outputs
out_traffic = widgets.Output()
out_recs    = widgets.Output()

# Create the Tab widget
tabs = widgets.Tab(children=[out_traffic, out_recs])
tabs.set_title(0, "Foot Traffic Data")
tabs.set_title(1, "Charger Recommendations")

# Populate the first tab with the foot traffic DataFrame
with out_traffic:
    display(df_foot_traffic.head(10))       # or any summary you want

# Prepare your recommendations DataFrame (e.g., top10_v3)
# Ensure it's in the notebook namespace:
recs_df = top10_v3[['location_name', 'final_score_v3']].reset_index(drop=True)

# Populate the second tab with the recommendations
with out_recs:
    display(recs_df)

# Finally, display the tabs
display(tabs)
